In [6]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

np.random.seed(42)

In [27]:
class PID:
    def __init__(self, kp, ki, kd):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.prev_error = 0
        self.Is = [0]

    def update(self, error, dt):
        P = self.kp * error
        self.Is.append(self.Is[-1] + self.ki * error * dt)
        D = self.kd * (error - self.prev_error) / dt
        self.prev_error = error
        return P + self.Is[-1] + D

In [28]:
class System:
    def __init__(self, x0, noise=0.1):
        self.x = x0
        self.noise = noise

    def update(self, u):
        self.x = self.x * 0.9 + u + np.random.randn() * self.noise
        return self.x

In [29]:
def run_pid(kp, ki, kd, noise, steps=500, target=2., dt=0.1):
    states = []
    control = []
    pid = PID(kp, ki, kd)
    init_state = 1
    system = System(init_state, noise=noise)

    for i in np.arange(0, steps, dt):
        error = target - system.x
        u = pid.update(error, dt)
        system.update(u)
        states.append(system.x)
        control.append(u)
    return np.mean((np.array(states[200:]) - target)**2)**0.5, states, control, pid.Is

In [30]:
error, states, control, Is = run_pid(0.1, 0.1, 0.01, 0)
px.line(x=np.arange(0, 500, 0.1), y=states).update_layout(title='States, error = {:.3f}'.format(error))

In [35]:
error, states, control, Is = run_pid(0.1, 0.1, 0.01, 0.01)
px.line(y=Is)

In [11]:
error, states, control = run_pid(0.1, 0.1, 0.01, 0.01)
px.line(x=np.arange(0, 500, 0.1), y=states).update_layout(title='States, error = {:.3f}'.format(error))

In [12]:
kps = np.arange(0.1, 1, 0.1)
kis = np.arange(0.1, 1, 0.1)
kds = np.arange(0, 0.05, 0.01)

res = np.empty((len(kps), len(kis), len(kds)))


for kp in kps:
    for ki in kis:
        for kd in kds:
            error, states, control = run_pid(kp, ki, kd, 0.01, dt=0.1)
            print('kp = {:.2f}, ki = {:.2f}, kd = {:.2f}, error = {:.4f}'.format(kp, ki, kd, error))
            res[int(kp*10-1), int(ki*10-1), int(kd*100-1)] = error

kp = 0.10, ki = 0.10, kd = 0.00, error = 0.0167
kp = 0.10, ki = 0.10, kd = 0.01, error = 0.0159
kp = 0.10, ki = 0.10, kd = 0.02, error = 0.0149
kp = 0.10, ki = 0.10, kd = 0.03, error = 0.0153
kp = 0.10, ki = 0.10, kd = 0.04, error = 0.0153
kp = 0.10, ki = 0.20, kd = 0.00, error = 0.0169
kp = 0.10, ki = 0.20, kd = 0.01, error = 0.0160
kp = 0.10, ki = 0.20, kd = 0.02, error = 0.0163
kp = 0.10, ki = 0.20, kd = 0.03, error = 0.0150
kp = 0.10, ki = 0.20, kd = 0.04, error = 0.0151
kp = 0.10, ki = 0.30, kd = 0.00, error = 0.0169
kp = 0.10, ki = 0.30, kd = 0.01, error = 0.0154
kp = 0.10, ki = 0.30, kd = 0.02, error = 0.0156
kp = 0.10, ki = 0.30, kd = 0.03, error = 0.0152
kp = 0.10, ki = 0.30, kd = 0.04, error = 0.0147
kp = 0.10, ki = 0.40, kd = 0.00, error = 0.0159
kp = 0.10, ki = 0.40, kd = 0.01, error = 0.0157
kp = 0.10, ki = 0.40, kd = 0.02, error = 0.0158
kp = 0.10, ki = 0.40, kd = 0.03, error = 0.0148
kp = 0.10, ki = 0.40, kd = 0.04, error = 0.0140
kp = 0.10, ki = 0.50, kd = 0.00, error =

In [20]:
i, j, k  = np.unravel_index(np.argmin(res), res.shape)
np.min(res, axis=(0, 1, 2)), (i+1)/10, (j+1)/10, kds[k]

(np.float64(0.009950317055984106),
 np.float64(0.9),
 np.float64(0.2),
 np.float64(0.04))

In [14]:
fig = sp.make_subplots(rows=1, cols=len(kds), subplot_titles=(["kd={i:.2f}".format(i=i) for i in kds]))
for i in range(5):
    fig.add_trace(go.Heatmap(z=res[:, :, i], x=kps, y=kis, reversescale=True, zmin=np.min(res, axis=(0, 1, 2)), zmax=np.max(res, axis=(0, 1, 2))), row=1, col=i+1)

fig.update_xaxes(title_text='kp',)
fig.update_yaxes(title_text='ki', row=1, col=1)


In [22]:
error, states, control = run_pid(0.9, 0.2, 0.0, noise=0.01, dt=0.1)
px.line(x=np.arange(0, 500, 0.1), y=states).update_layout(title='States, error = {:.5f}'.format(error))